# Handling Large Multi-Modal Payloads in AgentCore Runtime

## Overview

This tutorial demonstrates how Amazon Bedrock AgentCore Runtime handles large payloads up to 100MB, including multi-modal content such as Excel files and images. AgentCore Runtime is designed to process rich media content and large datasets seamlessly.

### Tutorial Details

|Information| Details|
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Large Payload & Multi-Modal Processing|
| Agent type          | Single         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Sonnet 3.7 |
| Tutorial components | Large File Processing, Image Analysis, Excel Data Processing |
| Tutorial vertical   | Data Analysis & Multi-Modal AI                                                   |
| Example complexity  | Intermediate                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK|

### Key Features

* **Large Payload Support**: Process files up to 100MB in size
* **Multi-Modal Processing**: Handle Excel files, images, and text simultaneously
* **Data Analysis**: Extract insights from structured data and visual content
* **Base64 Encoding**: Secure transmission of binary data through JSON payloads

## Prerequisites

* Python 3.10+
* AWS credentials configured
* Docker running
* Sample Excel file and image for testing

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Create Sample Data Files

Let's create sample Excel and image files to demonstrate large payload handling:

In [13]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# Create a large Excel file with sample sales data
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# Create a sample chart image
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# Draw a simple bar chart
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)
    y = 500 - height
    
    # Draw bar
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # Add labels (simplified without font)
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# Check file sizes
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB

print(f"Excel file size: {excel_size:.2f} MB")
print(f"Image file size: {image_size:.2f} MB")
print(f"Total payload size: {excel_size + image_size:.2f} MB")

Excel file size: 0.05 MB
Image file size: 0.01 MB
Total payload size: 0.05 MB


## Create Multi-Modal Agent

Let's create an agent that can process both Excel files and images from large payloads:

In [14]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Initialize the model and agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    You are a data analysis assistant that can process large Excel files and images.
    When given multi-modal data, analyze both the structured data and visual content,
    then provide comprehensive insights combining both data sources.
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Process large multi-modal payloads containing Excel data and images.
    
    Args:
        payload: Contains prompt, excel_data (base64), image_data (base64)
        context: Runtime context information
    
    Returns:
        str: Analysis results from both data sources
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== Large Payload Processing ===")
    print(f"Session ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel data size: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"Image data size: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel data {excel_data}")
    print(f"Image data {image_data}")
    print(f"=== Processing Started ===")
    # Decode base64 to bytes
    excel_bytes = base64.b64decode(excel_data)
    # Decode base64 to bytes
    image_bytes = base64.b64decode(image_data)
    
    # Enhanced prompt with data context
    enhanced_prompt = f"""{prompt}
    Please analyze both data sources and provide insights.
    """
    
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Writing multimodal_data_agent.py


## Setup Infrastructure and Deploy Agent

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="multimodal_data_agent"
)

Entrypoint parsed: file=/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_data_analysis/multimodal_data_agent.py, bedrock_agentcore_name=multimodal_data_agent
Configuring BedrockAgentCore agent: multimodal_data_agent
Generated Dockerfile: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_data_analysis/Dockerfile
Generated .dockerignore: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_data_analysis/.dockerignore
Keeping 'multimodal_data_agent' as default agent
Bedrock AgentCore configured: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_data_analysis/.bedrock_agentcore.yaml


In [18]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'multimodal_data_agent' to account 538213298977 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'multimodal_data_agent' to account 538213298977 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: multimodal_data_agent
✅ ECR repository available: 538213298977.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-multimodal_data_agent
Getting or creating execution role for agent: multimodal_data_agent
Using AWS region: us-east-1, account ID: 538213298977
Role name: AmazonBedrockAg

✅ Reusing existing ECR repository: 538213298977.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-multimodal_data_agent


✅ Reusing existing execution role: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-069e2d958e
✅ Execution role available: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-069e2d958e
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: multimodal_data_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-069e2d958e
Reusing existing CodeBuild execution role: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-069e2d958e
Using .dockerignore with 44 patterns
Uploaded source to S3: multimodal_data_agent/20250828-184630.zip
Updated CodeBuild project: bedrock-agentcore-multimodal_data_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.2s
🔄 PROVISIONING started (total: 5s)
✅ PROVISIONING completed in 5.2s
🔄 DOWNLOAD_SOURCE started (total: 11s)
✅ DOWNL

In [19]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Deployment status: {status}")

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: multimodal_data_agent


Final status: READY


## Test Large Multi-Modal Payloads

Now let's test the agent with large payloads containing both Excel data and images:

In [20]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# Encode files to base64
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# Create large payload
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 Processing large multi-modal payload...")
print(f"📋 Session ID: {session_id}")
print(f"📄 Excel size: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ Image size: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 Total payload: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# Invoke agent with large payload
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)
final_response = ""
for r in invoke_response['response']:
    final_response += r.decode("utf-8")
response_data = json.loads(final_response)
display(Markdown(response_data))

Invoking BedrockAgentCore agent 'multimodal_data_agent' via cloud endpoint


📊 Processing large multi-modal payload...
📋 Session ID: 06df269e-263c-4c5e-a721-77c9672f67ac
📄 Excel size: 0.06 MB
🖼️ Image size: 0.01 MB
📦 Total payload: 0.07 MB



# Sales Data Analysis

## Overview of the Dataset

I've analyzed your sales data from January 2023 to February 2023, which contains 1,000 records with hourly sales data for different products across various regions. The dataset includes information on product types (Gadget X, Gadget Y, Widget A, Widget B, Widget C), sales quantities, revenue, regions (East, West, North, South), and customer IDs.

## Product Performance Analysis

Looking at the chart image and correlating it with the Excel data:

1. **Product Sales Distribution**:
   - The chart shows four product categories with different sales volumes
   - The blue bar (tallest) appears to represent approximately 320 units
   - The coral/red bar shows around 250 units
   - The teal bar shows about 180 units
   - The light green bar shows approximately 150 units

2. **Excel Data Product Analysis**:
   - **Gadget X**: Total sales of 48,979 units (24.5% of total sales)
   - **Gadget Y**: Total sales of 41,399 units (20.7% of total sales)
   - **Widget A**: Total sales of 36,597 units (18.3% of total sales)
   - **Widget B**: Total sales of 36,726 units (18.4% of total sales)
   - **Widget C**: Total sales of 36,162 units (18.1% of total sales)

## Regional Performance

Based on the Excel data:

1. **Sales by Region**:
   - **North**: 51,735 units (25.9%)
   - **East**: 50,531 units (25.3%)
   - **South**: 49,479 units (24.7%)
   - **West**: 48,118 units (24.1%)

   This shows a relatively even distribution across all regions with North performing slightly better.

2. **Revenue by Region**:
   - **North**: $673,458,457 (26.6%)
   - **East**: $633,948,036 (25.1%)
   - **West**: $631,281,314 (25.0%)
   - **South**: $590,005,015 (23.3%)

   The North region generates the highest revenue despite only having a slight lead in unit sales.

## Time-Based Analysis

1. **Daily Sales Trends**:
   - Average daily sales: 6,449 units
   - Highest sales day: January 10 (7,136 units)
   - Lowest sales day: January 1 (5,237 units)

2. **Hourly Patterns**:
   - Peak sales hours: 9:00-11:00 AM and 4:00-6:00 PM
   - Lowest sales hours: 2:00-4:00 AM

## Price and Revenue Analysis

1. **Average Revenue per Unit**:
   - **Widget C**: $2,379 per unit (highest)
   - **Widget A**: $2,330 per unit
   - **Widget B**: $2,276 per unit
   - **Gadget Y**: $2,267 per unit
   - **Gadget X**: $2,206 per unit (lowest)

2. **Total Revenue by Product**:
   - **Gadget X**: $108,065,116 (21.4%)
   - **Widget C**: $86,032,257 (17.1%)
   - **Widget A**: $85,273,309 (16.9%)
   - **Widget B**: $83,586,083 (16.6%)
   - **Gadget Y**: $93,862,435 (18.6%)

## Key Insights

1. **Product Performance**: Gadget X is the highest-selling product by volume (as shown in the chart's blue bar), but Widget C generates more revenue per unit.

2. **Regional Balance**: The sales distribution across regions is fairly balanced, with North leading by a small margin. This suggests effective market penetration across all territories.

3. **Price Optimization Opportunity**: There's an inverse relationship between unit price and sales volume for some products. For example, Widget C has the highest revenue per unit but lower overall sales volume.

4. **Revenue vs. Volume**: While Gadget X leads in sales volume, its revenue per unit is the lowest, suggesting potential for price optimization to increase overall profitability.

5. **Time Patterns**: The data shows clear daily and hourly sales patterns that could be leveraged for targeted marketing campaigns or inventory management.

## Recommendations

1. **Product Strategy**: Consider increasing marketing for Widget C in regions where it performs best, as it offers the highest revenue per unit.

2. **Pricing Review**: Analyze the pricing strategy for Gadget X to potentially increase margins without significantly affecting sales volume.

3. **Regional Focus**: While regions are performing relatively evenly, the North region generates higher revenue per sale, suggesting potential for premium product positioning in this market.

4. **Time-Based Marketing**: Align marketing efforts with peak sales hours (9:00-11:00 AM and 4:00-6:00 PM) to maximize impact.

5. **Customer Segmentation**: Analyze high-value customers (those purchasing high-margin products) to identify common characteristics for targeted marketing.

## Cleanup Resources

In [21]:
import boto3

# Clean up AWS resources
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# Delete AgentCore Runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# Delete ECR repository
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Clean up local files
# os.remove('large_sales_data.xlsx')
# os.remove('sales_chart.png')

print("✅ Cleanup completed!")

✅ Cleanup completed!


# Congratulations!

You have successfully demonstrated handling large multi-modal payloads with Amazon Bedrock AgentCore Runtime!

## What you've learned:

### Large Payload Processing
* **100MB Support**: AgentCore Runtime can handle payloads up to 100MB
* **Base64 Encoding**: Secure transmission of binary data through JSON payloads
* **Efficient Processing**: Runtime optimized for large data processing

### Multi-Modal Capabilities
* **Excel Analysis**: Processing structured data from spreadsheets
* **Image Processing**: Analyzing visual content and charts
* **Combined Analysis**: Correlating insights from multiple data types

### Key Benefits
* **Rich Data Processing**: Handle complex, multi-format datasets
* **Scalable Architecture**: Runtime designed for large workloads
* **Tool Integration**: Custom tools for specialized data processing
* **Enterprise Ready**: Secure handling of sensitive business data

This demonstrates AgentCore Runtime's capability to handle enterprise-scale data processing tasks with multiple data modalities, making it ideal for complex business intelligence and data analysis applications.